In [1]:
# # stdlib
# import os

# os.environ["ORCHESTRA_DEPLOYMENT_TYPE"] = "remote"
# os.environ["DEV_MODE"] = "True"
# os.environ["TEST_EXTERNAL_REGISTRY"] = "k3d-registry.localhost:5800"

In [2]:
# isort: off
# stdlib
from os import environ as env
import os
import sys

# syft absolute
import syft as sy
# from syft import get_helpers  # noqa: F401
from syft.util.util import find_base_dir_with_tox_ini, get_caller_file_path, is_interpreter_jupyter


def add_helper_path_to_python_path() -> None:
    current_path = "."

    # jupyter uses "." which resolves to the notebook
    if not is_interpreter_jupyter():
        # python uses the file which has from syft import test_settings in it
        import_path = get_caller_file_path()
        if import_path:
            current_path = import_path

    base_dir = find_base_dir_with_tox_ini(current_path)
    notebook_helper_path = os.path.join(base_dir, "notebooks/scenarios/bigquery/upgradability/0.9.1_helpers")
    sys.path.append(notebook_helper_path)


add_helper_path_to_python_path()

# third party
from email_helpers import get_email_server
# isort: on

/home/teo/OpenMined/PySyft


# Launch & login

In [3]:
# when in k8s these are the default values
ROOT_EMAIL = "admin@bigquery.org"
ROOT_PASSWORD = "bqpw"

In [4]:
# in case we are not in k8s we set them here for orchestra to use
env["DEFAULT_ROOT_EMAIL"] = ROOT_EMAIL
env["DEFAULT_ROOT_PASSWORD"] = ROOT_PASSWORD

In [5]:
server = sy.orchestra.launch(
    name="bigquery-high",
    dev_mode=True,
    server_side_type="high",
    reset=True,
    port="8080",
    n_consumers=1,  # How many workers to be spawned
    create_producer=True,  # Can produce more workers
)

Autoreload enabled
Starting bigquery-high server on 0.0.0.0:8080
Found `reset=True` in the launch configuration. Resetting the server...


INFO:     Will watch for changes in these directories: ['/home/teo/OpenMined/PySyft/packages/syft/src/syft']
INFO:     Uvicorn running on http://0.0.0.0:8080 (Press CTRL+C to quit)
INFO:     Started reloader process [64107] using WatchFiles


Waiting for server to start...WARN: private key is based on server name: bigquery-high in dev_mode. Don't run this in production.
Using SQLiteDBConfig and sqlite:////tmp/syft/fbdf5a287e58454cbbd3fac4ad744d37/db/fbdf5a287e58454cbbd3fac4ad744d37_json.db


INFO:     Starting server with settings: {'name': 'bigquery-high', 'server_type': <ServerType.DATASITE: 'datasite'>, 'server_side_type': <ServerSideType.HIGH_SIDE: 'high'>, 'deployment_type': <DeploymentType.PYTHON: 'python'>, 'processes': 1, 'reset': False, 'dev_mode': True, 'enable_warnings': False, 'in_memory_workers': True, 'queue_port': None, 'create_producer': True, 'n_consumers': 1, 'association_request_auto_approval': False, 'background_tasks': False, 'db_config': None, 'db_url': None} and worker class: <class 'syft.server.datasite.Datasite'>


Adding producer for queue: api_call on: tcp://localhost:53903


KeyboardInterrupt: 

INFO:     Started server process [64121]
INFO:     Waiting for application startup.
INFO:     Application startup complete.
INFO:     Stopping reloader process [64107]


In [6]:
email_server, smtp_server = get_email_server(reset=True)

> Starting SMTPTestServer on: 0.0.0.0:9025


In [7]:
root_client = sy.login(
    url="http://localhost:8080", email=ROOT_EMAIL, password=ROOT_PASSWORD
)

> Starting SMTPTestServer on: 0.0.0.0:9025


Logged into <bigquery-high: High side Datasite> as <admin@bigquery.org>


SyftWarning: You are using a default password. Please change the password using `[your_client].account.set_password([new_password])`.

# Create new admin client

In [8]:
ADMIN_EMAIL, ADMIN_PW = "admin2@bigquery.org", "bqpw2"

In [9]:
# create
root_client.register(
    name="second admin", email=ADMIN_EMAIL, password=ADMIN_PW, password_verify=ADMIN_PW
)
# update role
new_user_id = root_client.users.search(email=ADMIN_EMAIL)[0].id
root_client.users.update(uid=new_user_id, role="admin")

```python
class UserView:
  id: str = 57df96fef9fb423998953a43699d54ae
  name: str = "second admin"
  email: str = "admin2@bigquery.org"
  institution: str = None
  website: str = None
  role: str = ServiceRole.ADMIN
  notifications_enabled: str = {: True, : False, : False, : False}

```

# We cannot delete the root client

In [10]:
high_client = sy.login(
    url="http://localhost:8080", email=ADMIN_EMAIL, password=ADMIN_PW
)

Logged into <bigquery-high: High side Datasite> as <admin2@bigquery.org>


In [11]:
root_admin_id = root_client.users.search(email=ROOT_EMAIL)[0].id

In [12]:
with sy.raises(sy.SyftException):
    high_client.users.delete(root_admin_id)

# Create ephemeral admin and delete it

In [13]:
# register 2nd new admin (to delete)
user_email, user_pw = "admin3@bigquery.org", "bqpw3"
## create
root_client.register(
    name="x", email=user_email, password=user_pw, password_verify=user_pw
)
## update role
new_user_id2 = root_client.users.search(email=user_email)[0].id
root_client.users.update(uid=new_user_id, role="admin")

```python
class UserView:
  id: str = 57df96fef9fb423998953a43699d54ae
  name: str = "second admin"
  email: str = "admin2@bigquery.org"
  institution: str = None
  website: str = None
  role: str = ServiceRole.ADMIN
  notifications_enabled: str = {: True, : False, : False, : False}

```

In [14]:
root_client.users.delete(new_user_id2)

<UID: c76d32b984344bef903385311a39fd49>

# Cleanup

In [15]:
smtp_server.stop()

> Stopping SMTPTestServer


In [16]:
server.land()

Stopping bigquery-high


INFO:     Shutting down
INFO:     Waiting for application shutdown.
INFO:     Application shutdown complete.
INFO:     Finished server process [362383]


INFO:     Stopping reloader process [362369]
